In [18]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# PointNet Model

Each datapoint consist of a set $\{x_i\}_{i=1}^k$ where $x_i = [\phi, \theta, depth, v]$. However the number $x_i$'s might differ between datapoints.

In [37]:
import torch.nn as nn


class MLP(nn.Module):

    def __init__(self, sizes):

        super(MLP, self).__init__()
        self.sizes = sizes
        self.activation = nn.ReLU()

        self.mlp = nn.Sequential(
            *(nn.Sequential(
                nn.Conv1d(self.sizes[i], self.sizes[i + 1], 1),
                nn.BatchNorm1d(self.sizes[i + 1]),
                self.activation
            ) for i in range(len(self.sizes) - 1))
        )

    def forward(self, x):
        return self.mlp(x)
    
# Check invariance
x = torch.randn(32, 3, 251).to(device)
perm = torch.randperm(251)
model = MLP([3, 64, 64]).to(device)
torch.all(torch.abs(model(x)[:, :, perm]-model(x[:, :, perm])) < 1e-6), "Permutation invariance failed"
    
class T_Net(nn.Module):
    
    def __init__(self, input_size=3):
        super(T_Net, self).__init__()
        
        self.input_size = input_size
        
        self.mlp1 = MLP([self.input_size, 64, 128, 1024])
        
        self.mlp2 = MLP([1024, 256, 128])
        
        self.mlp_out = nn.Sequential(
            nn.Conv1d(128, 128, 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, self.input_size ** 2, 1),
        )
        
    def forward(self, x):
        
        x = self.mlp1(x)
        x = torch.max(x, dim=-1)[0].unsqueeze(-1)
        x = self.mlp2(x)
        x = torch.max(x, dim=-1)[0].unsqueeze(-1)
        x = self.mlp_out(x)
        x = x.view(-1, self.input_size, self.input_size)
        
        return x
    
# Check Invariance
model = T_Net().to(device)
torch.all(torch.abs(model(x)-model(x[:, :, perm])) < 1e-6), "Permutation invariance failed"


class PointNet(nn.Module):
    def __init__(self, input_size=3, num_classes=10, dropout=0.3):
        super(PointNet, self).__init__()

        self.input_size = input_size
        self.num_classes = num_classes
        self.dropout = dropout

        self.input_transform = T_Net(input_size=self.input_size)
        self.feature_transform = T_Net(input_size=64)

        self.mlp1 = MLP([self.input_size, 64, 64])

        self.mlp2 = MLP([64, 64, 128, 1024])

        self.mlp_out = nn.Sequential(
            nn.Conv1d(1024, 512, 1),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Conv1d(512, 256, 1),
            nn.Dropout(self.dropout),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(256, self.num_classes, 1),
        )

    def forward(self, x, mask=None):

        if mask is None:
            mask = torch.ones(x.shape[0], x.shape[2]).to(device)  # mask: (B, # nodes)

        x = (x.transpose(1, 2) @ self.input_transform(x)).transpose(1, 2)
        x = self.mlp1(x)
        feature_transform = self.feature_transform(x)
        x = (x.transpose(1, 2) @ feature_transform).transpose(1, 2)
        x = self.mlp2(x)  # x: (B, # features, # Nodes), 1 if to keep else 0
        x = x.transpose(1, 2) - (1 - mask.unsqueeze(-1)) * torch.max(x)  # mask out nodes
        x = torch.max(x.transpose(1, 2), dim=-1)[0].unsqueeze(-1)
        x = self.mlp_out(x).squeeze(-1)

        return x, feature_transform

model = PointNet(dropout=0.0).to(device)
x = torch.randn(32, 3, 251).to(device)
assert model(x)[0].shape, "Model output shape is incorrect"

# Check Invariance
torch.all(torch.abs(model(x)[0]-model(x[:, :, perm])[0]) < 1e-6), "Permutation equivariance failed"
del model, x

# Load Data

In [38]:
from load_data import load_data
import numpy as np

train_data, train_labels = load_data()
val_index = np.random.choice(len(train_data), int(len(train_data) * 0.1), replace=False)
val_data = [train_data[i] for i in val_index]
val_labels = [train_labels[i] for i in val_index]
train_data = [train_data[i] for i in range(len(train_data)) if i not in val_index]
train_labels = [train_labels[i] for i in range(len(train_labels)) if i not in val_index]
test_data, _ = load_data('test')

def remove_nans(data):
    new_data = []
    for d in data:
        nan_index = np.where(d==-1)[0]
        min_nan = nan_index.min()
        new_data.append(d[:min_nan])

    return new_data

train_data = remove_nans(train_data)
val_data = remove_nans(val_data)
test_data = remove_nans(test_data)

means = np.concatenate(train_data).mean(axis=0)
stds = np.concatenate(train_data).std(axis=0)

train_data = [(d-means)/stds for d in train_data]
val_data = [(d-means)/stds for d in val_data]
test_data = [(d-means)/stds for d in test_data]

In [39]:
from torch.utils.data import Dataset, DataLoader

class PointCloudDataset(Dataset):

    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        label = self.labels[index] if self.labels is not None else None
        return torch.from_numpy(self.data[index]).to(torch.float32), label

    def __len__(self):
        return len(self.data)
    
train_dataset = PointCloudDataset(train_data, train_labels)
val_dataset = PointCloudDataset(val_data, val_labels)
test_dataset = PointCloudDataset(test_data)

def collate_fn(batch):
    lengths = [len(b[0]) for b in batch]
    max_length = max(lengths)
    paddings = [max_length - l for l in lengths]
    batch = [(np.pad(b[0], ((0, p), (0, 0))), b[1]) for b, p in zip(batch, paddings)]
    data, labels = zip(*batch)
    data = np.stack(data)
    labels = np.stack(labels)
    mask = np.ones(data.shape[:-1], dtype=np.float32)
    mask[np.where(data[:, :, 0] == 0)] = 0
    return torch.tensor(data).to(torch.float32), torch.tensor(labels).to(torch.int64) if labels[0] is not None else None, torch.tensor(mask).to(torch.float32)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Training

In [40]:
model = PointNet(input_size=4, num_classes=2, dropout=0.3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
loss = nn.CrossEntropyLoss()

In [41]:
def validate(model, val_loader):

    model.eval()
    total_loss = 0.0
    TP, TN, FP, FN = 0, 0, 0, 0

    for data, labels, mask in val_loader:

        data = data.to(device)
        labels = labels.to(device)
        preds, mat = model(data.transpose(1, 2), mask.to(device))
        total_loss += loss(preds, labels).item() + 0.001 * torch.norm(torch.eye(64).to(device) - mat.transpose(1, 2) @ mat, p='fro').item()
        preds = preds.argmax(dim=-1)
        TP += ((preds == 1) & (labels == 1)).sum().item()
        TN += ((preds == 0) & (labels == 0)).sum().item()
        FP += ((preds == 1) & (labels == 0)).sum().item()
        FN += ((preds == 0) & (labels == 1)).sum().item()

    total_samples = TP + TN + FP + FN    
    accuracy = (TP + TN) / total_samples
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    total_loss = total_loss / len(val_loader)

    return total_loss, accuracy, precision, recall, f1

In [42]:

num_epochs = 50
train_losses = []
val_scores = []
best_f1_val = 0.0

for epoch in range(num_epochs):
    epoch_loss = 0
    for data, labels, mask in train_loader:
        optimizer.zero_grad()
        output, mat = model(data.transpose(1, 2).to(device), mask.to(device))
        loss_val = loss(output, labels.to(device)) + 0.001 * torch.norm(torch.eye(64).to(device) - mat.transpose(1, 2) @ mat, p='fro')
        loss_val.backward()
        optimizer.step()
        epoch_loss += loss_val.item()
    epoch_loss /= len(train_loader)
    train_losses.append(epoch_loss)
    val_scores.append(validate(model, val_loader))
    if val_scores[-1][-1] > best_f1_val:
        best_f1_val = val_scores[-1][-1]
        torch.save(model.state_dict(), 'best_model.pth')
    scheduler.step()
    print(f'Epoch: {epoch+1}, Train Loss: {epoch_loss/len(train_loader):.2e}, Val Loss: {val_scores[-1][0]:.2e}, Val F1: {val_scores[-1][-1]:.2f}')

model.load_state_dict(torch.load('best_model.pth'))


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_losses, label='Train Loss')
plt.plot([s[0] for s in val_scores], label='Val Loss')
plt.yscale('log')
plt.legend()
plt.show()

plt.plot([s[1] for s in val_scores], label='Accuracy')
plt.plot([s[2] for s in val_scores], label='Precision')
plt.plot([s[3] for s in val_scores], label='Recall')
plt.plot([s[4] for s in val_scores], label='F1')
plt.legend()
plt.show()

# Get Predictions

In [ ]:
predictions = []
model.eval()
with torch.no_grad():
    for data, _, mask in test_loader:
        output = model(data.to(device).transpose(1, 2), mask.to(device))[0]
        predictions.append(output.argmax(dim=-1))
predictions = torch.cat(predictions).cpu().numpy()

# Save submission

In [ ]:
import shutil, os

os.makedirs('submission', exist_ok=True)

with open('submission/submission.csv', 'w') as f:
    f.write('label\n')
    for p in predictions:
        f.write(f'{p}\n')

shutil.copyfile('point_net.ipynb', 'submission/original_notebook.ipynb')
shutil.make_archive('submission', 'zip', 'submission')